In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,57,2024-02-27,-1,119.178571,98.978571,0.556000,12.025000,27.566071,0.191375,119.135714,...,11.747368,19.880702,0.175333,111.335088,0,2,1,2,10,2024
1,58,2024-02-27,-1,118.596491,99.756140,0.563140,10.738596,22.198246,0.200965,118.887719,...,11.569643,24.430357,0.187411,117.271429,0,2,1,2,7,2024
2,57,2024-02-27,-4,112.500000,97.089286,0.533946,10.669643,24.800000,0.177286,115.176786,...,12.427586,26.239655,0.223414,114.437931,0,2,1,3,3,2024
3,58,2024-02-27,-1,117.333333,97.994737,0.536386,10.085965,24.828070,0.241351,119.342105,...,10.582456,24.350877,0.195053,123.250877,0,2,1,2,23,2024
4,59,2024-02-27,1,118.017241,99.431034,0.545466,13.093103,29.029310,0.212724,117.653448,...,10.796491,27.731579,0.217965,119.464912,0,2,1,2,29,2024
5,59,2024-02-27,-2,116.086207,98.001724,0.555569,11.405172,25.634483,0.209052,118.298276,...,11.077193,29.547368,0.205018,119.515789,0,2,1,2,19,2024
6,59,2024-02-27,-4,112.051724,101.068966,0.528431,12.489655,22.665517,0.172448,110.589655,...,12.807018,23.142105,0.220789,116.871930,0,2,1,2,27,2024
7,58,2024-02-27,1,108.052632,97.447368,0.528000,11.900000,22.010526,0.168333,110.561404,...,11.117241,21.846552,0.221483,120.386207,0,2,1,2,5,2024
8,57,2024-02-27,-5,112.839286,99.851786,0.537214,13.019643,24.994643,0.191982,112.483929,...,10.654386,24.454386,0.187702,115.492982,0,2,1,3,9,2024
9,58,2024-02-27,-1,113.140351,98.443860,0.527842,11.196491,24.563158,0.205912,113.961404,...,10.519298,20.552632,0.205965,121.689474,0,2,1,2,11,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,1
1,2024-02-27,DAL,CLE,2.500000,1.555556,0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0
5,2024-02-27,NOP,NYK,1.704225,2.200000,0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0
8,2024-02-27,DET,CHI,5.550000,1.160000,0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,1
1,2024-02-27,DAL,CLE,2.500000,1.555556,0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0
5,2024-02-27,NOP,NYK,1.704225,2.200000,0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0
8,2024-02-27,DET,CHI,5.550000,1.160000,0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,0
1,2024-02-27,DAL,CLE,2.500000,1.555556,0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0
5,2024-02-27,NOP,NYK,1.704225,2.200000,0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0
8,2024-02-27,DET,CHI,5.550000,1.160000,0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,1
1,2024-02-27,DAL,CLE,2.500000,1.555556,0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0
5,2024-02-27,NOP,NYK,1.704225,2.200000,0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0
8,2024-02-27,DET,CHI,5.550000,1.160000,0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,1.0
1,2024-02-27,DAL,CLE,2.500000,1.555556,1.0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0.0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0.0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0.0
5,2024-02-27,NOP,NYK,1.704225,2.200000,1.0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0.0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0.0
8,2024-02-27,DET,CHI,5.550000,1.160000,0.0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-27,GSW,WAS,1.160000,5.550000,1.0
1,2024-02-27,DAL,CLE,2.500000,1.555556,1.0
2,2024-02-27,BRK,ORL,4.100000,1.253165,0.0
3,2024-02-27,PHI,BOS,6.250000,1.133333,0.0
4,2024-02-27,UTA,ATL,2.100000,1.769231,0.0
5,2024-02-27,NOP,NYK,1.704225,2.200000,1.0
6,2024-02-27,SAS,MIN,7.750000,1.095238,0.0
7,2024-02-27,CHO,MIL,9.500000,1.068966,0.0
8,2024-02-27,DET,CHI,5.550000,1.160000,0.0
9,2024-02-27,HOU,OKC,4.800000,1.200000,0.0
